In [1]:
import faiss
import os
import numpy as np
import pandas as pd

import torch
from torch import Tensor
from torchvision import models

from torchvision.transforms import Compose, transforms
from PIL import Image
import cv2
import sqlite3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
DATASET = "/nethome/kravicha3/aryan/project/dataset/Reddit_Provenance_Datasets/data/"
FEATURES_PATH = ""
INDEX_PATH = ""

# Parameters to change

1) Features extracted from ResNet-50  
    - different layers  
    - different models  
    - bigger features  
    - SIFT or SURF features  
2) Different distance metrics  
    - L1 or L2 norm  
3) Different Indexing  
    - HNSW  
    - OPQ  
    - IVF  
4) Non linearity using score = 1-tanh(distance)  
5) Using decompistions using PCA  


In [ ]:
from torchvision.models import MobileNet_V3_Large_Weights
# model = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights)
model = models.resnet50(pretrained=True, progress=False)
for param in model.parameters():
    param.requires_grad = False
model.fc = torch.nn.Identity()
model.to(device)
model.eval()

In [ ]:
def transform(images: np.ndarray):
    transformed = [transforms.ToTensor()]
    composed = Compose(transformed)
    return composed(Image.fromarray(images[:, :, ::-1])).unsqueeze(0)

def as_numpy(val: Tensor) -> np.ndarray:
    return val.detach().cpu().numpy()

def read_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        cap = cv2.VideoCapture(image_path)
        ret, img = cap.read()
        cap.release()
    return img

def model_output(image_path):
    img = read_image(image_path)
    imgt = transform(img)
    imgt = imgt.to(device)
    with torch.no_grad():
        inference = as_numpy(model(torch.unsqueeze(imgt[0], 0)))
    return inference

### Getting Images Features
1) Features extracted from ResNet-50  
    - different layers  
    - different models  
    - bigger features  
    - SIFT or SURF features 
2) Using decompistions using PCA

In [ ]:
from torchvision.models.feature_extraction import get_graph_node_names
nodes, _ = get_graph_node_names(model)
print('')

In [ ]:
def get_image_features(start_path = '.'):
    df = pd.DataFrame(columns=['img_name', 'dir', 'features'])
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
        
            if (fp.endswith('.jpg') or fp.endswith('.png') or fp.endswith('.mp4')):
                features = model_output(fp)
                df.loc[len(df.index)] = [f, dirpath, features]
    return df

In [ ]:
reddit_resnet_df = get_image_features(DATASET)

In [ ]:
reddit_resnet_df

In [ ]:
reddit_resnet_df.to_pickle('./tune_result/reddit_resnet_df.pkl')

In [2]:
df = pd.read_pickle('./tuning/tune_result/reddit_resnet_df.pkl')

In [6]:
df['features'][0]

array([[0.7295329 , 0.30392087, 0.7336758 , ..., 0.10279972, 0.1377843 ,
        0.65775776]], dtype=float32)

### Creating new Faiss Index
1) Different Faiss indexes:  
    - IndexFlatL2  
    - IndexIVFFlat  

In [ ]:
index = faiss.IndexFlatL2(dim)
index.train(features)